In [1]:
import pandas as pd
import requests
import os

vote_count_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv'
vote_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] Votelog.csv'


In [18]:
vote_count_df = pd.read_csv(vote_count_fpath)
vote_count_df.columns = vote_count_df.iloc[0]
vote_count_df = vote_count_df.iloc[1:]

vote_df = pd.read_csv(vote_fpath)


In [4]:
import re


def get_filename(url):
  return re.sub('^.*/', '', url)
  pass
  
get_filename('http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20190823113634A11.pdf')

'20190823113634A11.pdf'

In [28]:
from time import sleep


def download_pdf(url, save_dir):
  fpath = os.path.join(save_dir, get_filename(url))
  if os.path.isfile(fpath): 
    return fpath
  response = requests.get(url)
  sleep(200)
  if response.status_code != 200:
    return
  with open(fpath, 'wb') as fp:
    fp.write(response.content)
  return fpath

In [6]:
pdf_index = {}

In [34]:
def vote_of(vote_id):
  mask = vote_count_df[vote_id] != '-'
  cols = ['id', 'name', 'lastname', 'party', vote_id]
  return vote_count_df[cols][mask]

In [36]:
vote_id_col = (vote_df.columns[2])
for id, url in vote_df[[vote_id_col, 'ลิงค์ไปที่เอกสาร 1']].values:
  if isinstance(id, float) or not url.endswith('.pdf'): continue
  print('inprocess...', url)
  if id in vote_count_df.columns:
    fpath = download_pdf(url, 'test/pdf')
    if fpath is None: continue
    pdf_index[url] = fpath
    csv_fpath = re.sub('.pdf$', '.csv', fpath)
    vote_of(id).to_csv(csv_fpath, index=0)
  print('done')

inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20190606122944A1.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20190823113634A11.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20191008153956A1.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20190913143158A3.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20190917112227A5.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20191021111117A1.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20191021111117A3.pdf
done
inprocess... http://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20191122170454A1.pdf
done
inprocess... http://msbis.parli

In [12]:
pd.DataFrame(pdf_index.items())

,0,1
0,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20190823113634A11.pdf
1,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191008153956A1.pdf
2,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20190913143158A3.pdf
3,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20190917112227A5.pdf
4,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191021111117A1.pdf
5,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191021111117A3.pdf
6,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191122170454A1.pdf
7,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191128082252A1.pdf
8,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191206075729A1.pdf
9,http://msbis.parliament.go.th/ewtadmin/ewt/par...,test/pdf/20191219171121A1.pdf


In [1]:
from pdf2image import convert_from_path

In [3]:
import fitz

In [5]:
doc = fitz.open('test/pdf/20190823113634A11.pdf')

In [ ]:
from os import listdir


for file in listdir('test'):

In [10]:
mat = fitz.Matrix(3, 3)
doc[0].get_pixmap(matrix=mat).save("0.png")